<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [51]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [52]:
RANDOM_STATE = 42

## Подготовка

#### Загрузка данных

In [53]:
try:
    comments = pd.read_csv('../datasets/toxic_comments.csv')
except FileNotFoundError:
    comments = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

comments = comments.rename(columns={comments.columns[0]: 'index'})
comments = comments.set_index('index')

In [54]:
comments

,text,toxic
index,,
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0


#### Загрузка стоп-слов

In [55]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikitakhavkin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Предобработка & TF-IDF

In [56]:
corpus = comments['text'].values.astype('U')

X_train, X_test, y_train, y_test = train_test_split(
    corpus, comments['toxic'], test_size=0.2, random_state=RANDOM_STATE
)

In [57]:
vectorizer = TfidfVectorizer(stop_words=list(stopwords))

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

## Обучение

#### Обучение моделей <br>
Обучим 3 модели: логистическую регрессию, случайный лес и градиентный бустинг

In [58]:
models = [
    ("Logistic Regression", LogisticRegression(random_state=RANDOM_STATE)),
    ("Random Forest", RandomForestClassifier(random_state=RANDOM_STATE)),
    ("Gradient Boosting", GradientBoostingClassifier(random_state=RANDOM_STATE))
]

Цикл для обучения моделей и вывода метрики F1

In [59]:
for model_name, model in models:
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"F1-score for {model_name}: {f1:.4f}")

F1-score for Logistic Regression: 0.9501
F1-score for Random Forest: 0.9470
F1-score for Gradient Boosting: 0.9277


#### Промежуточный вывод <br>
Все модели показали значение метрики F1 выше 0.75 <br>
Лучшее значение метрики показала модель логистической регрессии - 0.9501

## Выводы

В ходе исследования были выполнены следующие шаги:
1. Загружены данные
2. Загружены стоп-слова
3. Проведена предобработка данных (токенизация)
4. Обучены 3 модели: логистическая регрессия, случайный лес и градиентный бустинг
5. Проведена оценка качества моделей по метрике F1
6. Вывод: лучшее значение метрики показала модель логистической регрессии - 0.9501

### Комментарий студента <br>
Привет, сначала пытался сделать проект с использованием BERT, но время обучения было очень долгим. Это всегда так или я неправильно выполнил предобработку / неправильно использовал библиотеку? <br>
И в целом какие плюсы от использования BERT в данной задаче? Метрика F1 была бы выше?

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны